In [1]:
import asf_search as asf
from RAiDER.losreader import get_orbit, read_ESA_Orbit_file
import isce3.ext.isce3 as isce
import xarray as xr
from shapely.geometry import Point
import datetime
import numpy as np
from hyp3lib import get_orb as fetch_orbit_file
import pandas as pd
from azimuth_timing import get_azimuth_timing_grid

# 1 Get Lon/Lat/Height Grid

In [2]:
gunw_path = 'S1-GUNW-A-R-064-tops-20210723_20210711-015000-00119W_00033N-PP-6267-v2_0_6.nc'
group = 'science/grids/imagingGeometry'
with xr.open_dataset(gunw_path, group=group, mode='r') as ds:
    res_x, res_y = ds.rio.resolution()
    # assuming ul corner centered
    lat = ds.latitudeMeta.data - res_y / 2.
    lon = ds.longitudeMeta.data - res_x / 2.
    hgt = ds.heightsMeta.data
lat

array([35.75, 35.65, 35.55, 35.45, 35.35, 35.25, 35.15, 35.05, 34.95,
       34.85, 34.75, 34.65, 34.55, 34.45, 34.35, 34.25, 34.15, 34.05,
       33.95, 33.85, 33.75, 33.65, 33.55, 33.45, 33.35, 33.25])

In [3]:
group = 'science/radarMetaData/inputSLC'
with xr.open_dataset(gunw_path, group=f'{group}/reference') as ds:
    slc_ids = ds['L1InputGranules'].data
    # Ensure non-empty and sorted by acq_time
    slc_ids = sorted(list(filter(lambda x: x, slc_ids)))
    #slc_start_time = get_start_time_from_slc_id(slc_ids[0])

In [5]:
slc_ids

['S1B_IW_SLC__1SDV_20210723T014947_20210723T015014_027915_0354B4_B3A9.zip']

# 2 Get orbit file from *time* and *center pos*

## Get SLC id

In [6]:
lat_center = np.mean(lat)
lon_center = np.mean(lon)
buffer_seconds = 60
time_center = datetime.datetime(2021, 7, 23, 1, 50, 0)
time_delta = datetime.timedelta(seconds=buffer_seconds)


point = Point(lon_center, lat_center)
results = asf.geo_search(intersectsWith=point.wkt,
                         processingLevel=asf.PRODUCT_TYPE.SLC,
                         start=time_center - time_delta,
                         end=time_center + time_delta,
                         maxResults=5
                        )

In [7]:
slc_id = results[0].properties['sceneName']

## Get Orbit File

In [8]:
orb_file, _ = fetch_orbit_file.downloadSentinelOrbitFile(slc_id)
orb_file

'S1B_OPER_AUX_RESORB_OPOD_20210723T045406_V20210723T000156_20210723T031926.EOF'

# 3 Generate Azimuth Timing

In [9]:
orb = get_orbit(orb_file, time_center, pad=600)
orb

In [10]:
X = get_azimuth_timing_grid(lon, lat, hgt, orb)
X

array([[['2021-07-23T01:50:26', '2021-07-23T01:50:26',
         '2021-07-23T01:50:26', ..., '2021-07-23T01:50:18',
         '2021-07-23T01:50:18', '2021-07-23T01:50:18'],
        ['2021-07-23T01:50:25', '2021-07-23T01:50:24',
         '2021-07-23T01:50:24', ..., '2021-07-23T01:50:17',
         '2021-07-23T01:50:17', '2021-07-23T01:50:16'],
        ['2021-07-23T01:50:23', '2021-07-23T01:50:23',
         '2021-07-23T01:50:23', ..., '2021-07-23T01:50:15',
         '2021-07-23T01:50:15', '2021-07-23T01:50:15'],
        ...,
        ['2021-07-23T01:49:49', '2021-07-23T01:49:49',
         '2021-07-23T01:49:49', ..., '2021-07-23T01:49:41',
         '2021-07-23T01:49:41', '2021-07-23T01:49:41'],
        ['2021-07-23T01:49:48', '2021-07-23T01:49:48',
         '2021-07-23T01:49:47', ..., '2021-07-23T01:49:40',
         '2021-07-23T01:49:39', '2021-07-23T01:49:39'],
        ['2021-07-23T01:49:46', '2021-07-23T01:49:46',
         '2021-07-23T01:49:46', ..., '2021-07-23T01:49:38',
         '2021-07